In [1]:
import numpy as np
import pandas as pd
import psycopg2
from scipy.stats import ks_2samp
import os
import random
from google.colab import auth
from tabulate import tabulate
try:
    os.mkdir("out")
except FileExistsError:
    pass

/usr/local/lib/python3.7/dist-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [35]:
auth.authenticate_user()

def run_query(query, project_id="mimic-project-324510"):
  return pd.io.gbq.read_gbq(
      query,
      project_id=project_id,
      dialect='standard')
  
def all_but_string(list_tup, combine="AND", variable="diag.long_title", equality="LIKE", then=False):
    # Makes a SQL string that enters variables not equal to some string
    out_str = ""
    initial_comment = True
    for i, str_ in enumerate(list_tup):
        if str_ != '':
            if not initial_comment:
                out_str = out_str + f"\n{combine} "
            if "'" not in str_:
                out_str = out_str + f"""LOWER ( {variable} ) {equality} '%{str_.lower()}%' """
                if then:
                    out_str = out_str + f"""THEN '{str_.lower()}'"""
            else:
                out_str = out_str + f'''LOWER ( {variable} ) {equality} "%{str_.lower()}%" '''
                if then:
                    out_str = out_str + f'''THEN "{str_.lower()}"'''
            initial_comment = False
    return out_str

def get_static_data_index(static_data, N_measr, N_unique, N_count, N_dicom):
    # Static data must contain columns "label", "dicom_id", "valuenum"
    # Count measurements and cut fewer than N_measr total (across DICOMs)
    counts = static_data.groupby(["label", "dicom_id"])["valuenum"].count()
    DICOM_COUNTS = counts.index.get_level_values("label").value_counts()
    idxs = DICOM_COUNTS[DICOM_COUNTS >= N_measr].index
    # Count unique measurement values and cut fewer than N_unique per label
    UNIQUE_COUNTS = static_data.groupby(["label"])["valuenum"].unique().apply(lambda x: len(x))
    idxs = idxs[idxs.isin(UNIQUE_COUNTS[UNIQUE_COUNTS >= N_unique].index)]
    # Update
    counts = counts[idxs]
    # Count measurements and cut fewer than N_measr and filter the number
    # of measurements associated with each DICOM to > N_dicom
    dicom_counts = counts[counts > N_count].index.get_level_values("label").value_counts()
    high_idxs = (dicom_counts[dicom_counts > N_dicom]).index
    low_idxs = idxs[~idxs.isin(high_idxs)]
    return idxs, high_idxs, low_idxs

def time_query(period, time_col="charttime"):
    # Period as int describes the number of days prior to the day of CXR
    if type(period) is not int:
        assert period in ("day", "stay"), "Period given as string type must be either 'day' or 'stay'."
        if period == "day":
            return f"d.AcquisitionDate = REPLACE ( CAST ( DATE ( a.{time_col} ) AS STRING ), '-', '' )"
        elif period == "stay":
            return f"d.AcquisitionDate between REPLACE ( CAST ( DATE ( icu.intime ) AS STRING ), '-', '' ) and REPLACE ( CAST ( DATE ( a.{time_col} ) AS STRING ), '-', '' )"
    return f"CAST ( d.AcquisitionDate AS FLOAT64 ) between CAST ( REPLACE ( CAST ( DATE ( a.{time_col} ) AS STRING ), '-', '' ) AS FLOAT64 ) - {period} and CAST ( REPLACE ( CAST ( DATE ( a.{time_col} ) AS STRING ), '-', '' ) AS FLOAT64 )"

Get overlapping patients that have been at the ICU

In [12]:
overlap_query = f"""
SELECT r.subject_id, i.hadm_id
FROM physionet-data.mimic_cxr.record_list r
JOIN physionet-data.mimic_icu.icustays i
ON r.subject_id = i.subject_id
JOIN physionet-data.mimic_cxr.dicom_metadata_string dicom
ON r.dicom_id = dicom.dicom
    AND dicom.AcquisitionDate between REPLACE ( CAST ( DATE ( i.intime ) AS STRING ), '-', '' ) and REPLACE ( CAST ( DATE ( i.outtime ) AS STRING ), '-', '' )
GROUP BY r.subject_id, i.hadm_id
"""
overlapping_subjects = run_query(overlap_query)
display(overlapping_subjects)

,subject_id,hadm_id
0,10021487,28998349
1,10082560,23284776
2,10144089,24171172
3,10148417,29867930
4,10190445,27005502
...,...,...
18316,16337794,24707597
18317,14574668,24152736
18318,17356318,25063767
18319,19209496,27405242


Find most common diagnoses in the overlapping dataset (ignoring the different icd versions)

In [13]:
N_diagnoses = 250
most_common_query = f"""
SELECT COUNT(i.long_title) as `total_cases`, i.long_title
from `physionet-data.mimic_hosp.d_icd_diagnoses` i JOIN `physionet-data.mimic_hosp.diagnoses_icd` d
ON d.icd_code = i.icd_code
    AND d.icd_version = i.icd_version
    AND d.hadm_id IN {tuple(overlapping_subjects.hadm_id.values)}
GROUP BY i.long_title
ORDER BY COUNT(i.long_title) DESC LIMIT {N_diagnoses}
"""
most_common = run_query(most_common_query)
display(most_common)
most_common.to_csv("out/most_common_diagnoses.csv", index=False)

,total_cases,long_title
0,5734,Unspecified essential hypertension
1,4860,Other and unspecified hyperlipidemia
2,4817,"Acute kidney failure, unspecified"
3,3973,"Congestive heart failure, unspecified"
4,3798,Atrial fibrillation
...,...,...
245,268,Other late effects of cerebrovascular disease
246,267,Unspecified hereditary and idiopathic peripher...
247,263,"Other and unspecified alcohol dependence, cont..."
248,262,Adrenal cortical steroids causing adverse effe...


In [14]:
count_diagnoses_query = lambda diagnosis: f"""
SELECT COUNT ( DISTINCT ( d.hadm_id ) ) as count, diag.long_title
from `physionet-data.mimic_hosp.diagnoses_icd` d
JOIN `physionet-data.mimic_hosp.d_icd_diagnoses` diag
ON d.icd_code = diag.icd_code
    AND d.icd_version = diag.icd_version
    AND d.hadm_id IN {tuple(overlapping_subjects.hadm_id.values)}
    AND {all_but_string([diagnosis], variable="diag.long_title", equality="LIKE")}
GROUP BY diag.long_title
ORDER BY COUNT ( DISTINCT ( d.hadm_id ) ) DESC
"""

In [25]:
diagnosis_ = "personal history"
count_diagnoses = run_query(count_diagnoses_query(diagnosis_))
count_ = count_diagnoses["count"].sum()
print(f"The total count of {diagnosis_} is {count_}")
display(count_diagnoses)

The total count of personal history is 15405


,count,long_title
0,3230,Personal history of tobacco use
1,1476,Personal history of nicotine dependence
2,1246,Personal history of transient ischemic attack ...
3,793,Personal history of venous thrombosis and embo...
4,690,Personal history of pulmonary embolism
...,...,...
136,1,Personal history of malignant carcinoid tumor ...
137,1,Personal history of malignant carcinoid tumor ...
138,1,Personal history of other (healed) physical in...
139,1,Personal history of nephrotic syndrome


### Possible diagnoses:

- Atelectasis, total cases: 202

- Cardiomegaly, total cases: 44
    - Heart failure is likely more frequently coded

- Diaphragmatic hernia, total cases: 316

- Infiltration, total cases: *not found*

- Mass (cancer)
    - Sarcoma, total cases: *very few in the chest region*

- Pleural thickening, total cases: *very few*

- Pleural effusion, total cases: 1361

- Pneumonia, total unique cases: 4686
  - Requires multiple modalities:
  - X-ray
  - Test results:
    - CRP
    - White bloodcells (leucocytes)
    - Nasopharynx
    - Test spit
  - Time series:
    - O2-levels
    - Pulse
    - Respiratory rate

- Pneumothorax, total cases: 467

- Pulmonary edema, total cases: 218

- Pulmonary fibrosis, total_cases: 250

- Pulmonary nodule, total cases: 316 


From "*Large Scale Automated Reading of Frontal and Lateral Chest X-Rays using Dual Convolutional Neural Networks*" (2018)

Find patients that fit the above diagnosis

In [26]:
diagnoses_string = [
    'Diaphragmatic hernia',
    'Pneumonia',
    'Pneumothorax',
    'Pleural effusion',
    'Pulmonary nodule',
]
exceptions = {
    diagnoses_string[i] : diagnoses_string[:i] + diagnoses_string[i+1:] for i in range(len(diagnoses_string))
}
# diagnoses_string = [
#     'Pleural effusion',
#     '',
# ]
# exceptions = {
#     diagnoses_string[0] : [
#         'Atelectasis',
#         'Cardiomegaly',
#         'Diaphragmatic hernia',
#         'Pneumonia',
#         'Pneumothorax',
#         'Pulmonary fibrosis',
#         'Pulmonary nodule',
#     ],
#     diagnoses_string[-1] : [
#         'Atelectasis',
#         'Cardiomegaly',
#         'Diaphragmatic hernia',
#         'Pneumonia',
#         'Pneumothorax',
#         'Pulmonary edema',
#         'Pleural effusion',
#         'Pulmonary fibrosis',
#         'Pulmonary nodule',
#     ]
# }

In [27]:
rare_disease_filter_N = 0

for diagnosis_ in diagnoses_string:
    if diagnosis_ != '':
        diag_variants = run_query(count_diagnoses_query(diagnosis_))
        exceptions[diagnosis_].extend(diag_variants[diag_variants["count"] < rare_disease_filter_N].long_title.to_list())

In [28]:
exceptions

{'Diaphragmatic hernia': ['Pneumonia',
  'Pneumothorax',
  'Pleural effusion',
  'Pulmonary nodule'],
 'Pleural effusion': ['Diaphragmatic hernia',
  'Pneumonia',
  'Pneumothorax',
  'Pulmonary nodule'],
 'Pneumonia': ['Diaphragmatic hernia',
  'Pneumothorax',
  'Pleural effusion',
  'Pulmonary nodule'],
 'Pneumothorax': ['Diaphragmatic hernia',
  'Pneumonia',
  'Pleural effusion',
  'Pulmonary nodule'],
 'Pulmonary nodule': ['Diaphragmatic hernia',
  'Pneumonia',
  'Pneumothorax',
  'Pleural effusion']}

# Sampling

In [29]:
balance_dataset = False
N = np.inf
random_state = 3

subj_query = lambda diagnosis, not_diagnosis: f"""
SELECT d.subject_id, d.hadm_id
from `physionet-data.mimic_hosp.diagnoses_icd` d
JOIN `physionet-data.mimic_hosp.d_icd_diagnoses` diag
ON d.icd_code = diag.icd_code
    AND d.icd_version = diag.icd_version
    AND d.hadm_id IN {tuple(overlapping_subjects.hadm_id.values)} {f'''
    AND ({
        all_but_string([diagnosis], variable="diag.long_title", equality="LIKE")
    })''' if diagnosis != '' else ''}
    WHERE NOT EXISTS (
        SELECT d2.hadm_id
        from `physionet-data.mimic_hosp.diagnoses_icd` d2
        JOIN `physionet-data.mimic_hosp.d_icd_diagnoses` diag2
        ON d2.icd_code = diag2.icd_code
            AND d2.icd_version = diag2.icd_version
            AND d.hadm_id = d2.hadm_id
            AND ({f'''{
                all_but_string(not_diagnosis, combine="OR", variable="diag2.long_title", equality="LIKE")
            } ''' if not_diagnosis != [''] else ''} {f'''
            {f" OR " if not_diagnosis != [''] else ''} {
                all_but_string(exceptions[diagnosis], combine="OR", variable="diag2.long_title", equality="LIKE")
            } ''' if exceptions[diagnosis] != [] else ''}) )
GROUP BY d.subject_id, d.hadm_id
ORDER BY d.hadm_id
"""

diags_tables = [run_query(subj_query(
    diagnoses_string[_],
    diagnoses_string[:_] + diagnoses_string[_+1:]
)) for _ in range(len(diagnoses_string))]
print(f"Balance: {[len(_) for _ in diags_tables]}")
N = min(*[len(_) for _ in diags_tables], N) # Ensure equal sample size
assert N != 0, f"Length of diagnoses are, respectively {[len(_) for _ in diags_tables]}"

if balance_dataset:
    sample = pd.concat(
        [_.sample(n=N, replace=False, random_state=random_state) for _ in diags_tables],
        ignore_index=True
    )
else:
    sample = pd.concat(
        [_.sample(frac=1., replace=False, random_state=random_state) for _ in diags_tables],
        ignore_index=True
    )

print(f"{100 * len(sample.hadm_id.unique())/len(sample)}% unique values, 100% suggests entirely separate populations")
sample = sample.sample(
    frac=1, replace=False, random_state = random_state
).reset_index(drop=True)
display(sample)

Balance: [201, 3438, 268, 733, 204]
1204.9751243781095% unique values, 100% suggests entirely separate populations


,subject_id,hadm_id
0,15985181,20334412
1,13366982,23172421
2,10793324,20567628
3,18190098,20394032
4,13273041,29356867
...,...,...
4839,15741464,24480630
4840,15031358,21343867
4841,16505030,27981003
4842,16800099,28762268


# CXR :
Combine *subject_id*, *study_id*, *dicom_id*, *AcquisitionDate*, image properties?  
Feature vectors linked to single X-rays.

Join tables **record_list** and **dicom_metadata_string**


# IV :
Combine with **CXR** for a given time window based on *AcquisitionDate* above.

Which features to extract?  
**Lab tests**, **chart events**


In [31]:
labels_query = f"""
SELECT d.subject_id, d.hadm_id, CASE WHEN {
    all_but_string([_ for _ in diagnoses_string if _ != ''], combine="WHEN", variable="diag.long_title", equality="LIKE", then=True)
    } ELSE 'None' END as long_title
FROM `physionet-data.mimic_hosp.diagnoses_icd` d
JOIN `physionet-data.mimic_hosp.d_icd_diagnoses` diag
ON d.icd_code = diag.icd_code
    AND d.icd_version = diag.icd_version
    AND d.hadm_id IN {tuple(sample.hadm_id.values)}
GROUP BY d.subject_id, d.hadm_id, CASE WHEN {
    all_but_string([_ for _ in diagnoses_string if _ != ''], combine="WHEN", variable="diag.long_title", equality="LIKE", then=True)
    } ELSE 'None' END
"""
labels_ = run_query(labels_query)
labels = labels_[labels_.long_title.isin([_.lower() for _ in diagnoses_string if _ != ''])]
labels = labels.append(labels_[~labels_.hadm_id.isin(labels.hadm_id)]).sample(frac=1, replace=False).reset_index(drop=True)
labels = sample.rename(columns={"hadm_id" : "hadm_id"}).merge(labels[["hadm_id", "long_title"]], on="hadm_id").sort_values("hadm_id")
display(labels)
labels.to_csv("out/labels.csv", index=False)

,subject_id,hadm_id,long_title
576,10236621,20004004,diaphragmatic hernia
1309,14583219,20004577,pleural effusion
310,19669999,20005479,diaphragmatic hernia
3725,17366128,20008724,pneumothorax
4447,12744745,20009335,pneumonia
...,...,...,...
112,10449297,29981093,pleural effusion
4349,13401524,29986642,pneumonia
2887,10983866,29994310,pneumonia
4019,16708802,29996606,pneumonia


In [32]:
diags_query = f"""
SELECT d.subject_id, d.hadm_id, diag.long_title
FROM `physionet-data.mimic_hosp.diagnoses_icd` d
JOIN `physionet-data.mimic_hosp.d_icd_diagnoses` diag
ON d.icd_code = diag.icd_code
    AND d.icd_version = diag.icd_version
    AND d.hadm_id IN {tuple(sample.hadm_id.values)}
GROUP BY d.subject_id, d.hadm_id, diag.long_title
"""
diags_ = run_query(diags_query)
diags_table = pd.concat((diags_, pd.Series(1, index=diags_.index, name="ones")), axis=1).pivot_table(
    index='hadm_id',
    columns='long_title',
    values='ones',
    fill_value=0,
)
hadm_diags = sample.hadm_id.apply(lambda x: diags_table.loc[x])
hadm_diags.index = sample.hadm_id
hadm_diags = hadm_diags.sort_index()
display(hadm_diags)
hadm_diags.to_csv("out/hadm_diags.csv")

long_title,(Idiopathic) normal pressure hydrocephalus,24 weeks gestation of pregnancy,37 weeks gestation of pregnancy,38 weeks gestation of pregnancy,Abdominal aneurysm without mention of rupture,"Abdominal aneurysm, ruptured","Abdominal aortic aneurysm, ruptured","Abdominal aortic aneurysm, without rupture","Abdominal or pelvic swelling, mass, or lump, unspecified site","Abdominal pain, epigastric",...,Wernicke's encephalopathy,Wet beriberi,Wheelchair dependence,Wheezing,Zoster without complications,"Zygomatic fracture, left side, initial encounter for closed fracture","Zygomatic fracture, right side, initial encounter for closed fracture","Zygomatic fracture, unspecified side, initial encounter for closed fracture",Zygomycosis [Phycomycosis or Mucormycosis],von Gierke disease
hadm_id,,,,,,,,,,,,,,,,,,,,,
20004004,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
20004577,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
20005479,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
20008724,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
20009335,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29981093,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
29986642,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
29994310,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Combine the info
**subject_id** in the list of patients with the diagnosis above,  
at the correct time, in the correct admission, between admission and discharge,  
with the correct image while in the ICU,  
with the correct radiology report.

In [33]:
image_icu_query = f"""
select
    a.subject_id,
    a.hadm_id,
    a.stay_id,
    b.dicom_id,
    d.AcquisitionDate as dicom_date,
    d.AcquisitionTime as dicom_time,
    b.path as dicom_path,
    s.path as study_path
from `physionet-data.mimic_icu.icustays` a
JOIN `physionet-data.mimic_cxr.record_list` b
ON a.subject_id = b.subject_id
    AND a.hadm_id IN {tuple(sample.hadm_id.values)}
JOIN `physionet-data.mimic_cxr.dicom_metadata_string` d
ON b.dicom_id = d.dicom
    AND d.ViewPosition = 'AP'
    AND d.AcquisitionDate between REPLACE ( CAST ( DATE ( a.intime ) AS STRING ), '-', '' ) and REPLACE ( CAST ( DATE ( a.outtime ) AS STRING ), '-', '' )
JOIN `physionet-data.mimic_cxr.study_list` s
ON b.study_id = s.study_id
ORDER BY hadm_id
"""
unique_images = run_query(image_icu_query)
display(unique_images)
unique_images.to_csv('out/images.csv', index=False)

,subject_id,hadm_id,stay_id,dicom_id,dicom_date,dicom_time,dicom_path,study_path
0,10236621,20004004,31718519,cd65a5f7-d10c6901-53c6b6f3-82601b13-afbdda25,21710628,160840.531,files/p10/p10236621/s53575978/cd65a5f7-d10c690...,files/p10/p10236621/s53575978.txt
1,19669999,20005479,32977919,01443be1-d424fb1a-6110423f-9b2a3b4f-d275af98,21480602,042251.437,files/p19/p19669999/s50891543/01443be1-d424fb1...,files/p19/p19669999/s50891543.txt
2,19669999,20005479,32977919,927ce6d1-2f7acc58-55042c9a-e688c114-129c4009,21480601,121446.343,files/p19/p19669999/s56328195/927ce6d1-2f7acc5...,files/p19/p19669999/s56328195.txt
3,17366128,20008724,37059438,be321c13-8ab85f5d-49bd822c-5e59f597-5178f216,21240226,081122.015,files/p17/p17366128/s52204648/be321c13-8ab85f5...,files/p17/p17366128/s52204648.txt
4,17366128,20008724,37059438,b0ec0b12-0b239642-cde1bd31-b9eee7ad-9eeed1b7,21240227,054900.703,files/p17/p17366128/s56691347/b0ec0b12-0b23964...,files/p17/p17366128/s56691347.txt
...,...,...,...,...,...,...,...,...
31119,16708802,29996606,34034988,7803e372-c84ecb49-427f2b1e-31736f69-74a62343,21301205,083818.875,files/p16/p16708802/s50161114/7803e372-c84ecb4...,files/p16/p16708802/s50161114.txt
31120,16708802,29996606,34034988,e6d3052a-9de5af50-2dd7cabd-7d85f1ee-1ef876ea,21301206,122854.921,files/p16/p16708802/s54400912/e6d3052a-9de5af5...,files/p16/p16708802/s54400912.txt
31121,16708802,29996606,34034988,46ef560e-476fbbbe-2c0ed8e7-0e189447-d93980b7,21301205,044626.203,files/p16/p16708802/s57449636/46ef560e-476fbbb...,files/p16/p16708802/s57449636.txt
31122,13478841,29999498,33874605,211b69be-01ae57d7-0774634f-18585752-f5d74b64,21880513,081601.734,files/p13/p13478841/s58035940/211b69be-01ae57d...,files/p13/p13478841/s58035940.txt


Grab values from mimic_icu.datetimeevents, -.chartevents, and -.procedureevents that overlap in time window with dicom_id-AquisitionDate+AquisitionTime

# Extract linear data
- Vital signs
- Lab tests

In [36]:
age = run_query(
f"""
select
    subject_id,
    hadm_id,
    age
from `physionet-data.mimic_derived.age`
order by hadm_id
"""
)
vital_signs = run_query(
f"""
select
    b.hadm_id,
    r.dicom_id,
    a.*
except (temperature_site)
from `physionet-data.mimic_derived.vitalsign` a
join `physionet-data.mimic_icu.icustays` b
on a.stay_id = b.stay_id
    and b.hadm_id IN {tuple(sample.hadm_id.values)}
JOIN `physionet-data.mimic_cxr.record_list` r
ON r.subject_id = b.subject_id
JOIN `physionet-data.mimic_cxr.dicom_metadata_string` d
ON d.dicom = r.dicom_id
    AND d.AcquisitionDate = REPLACE ( CAST ( DATE ( a.charttime ) AS STRING ), '-', '' )
order by b.hadm_id, a.charttime
"""
)

In [37]:
is_nan = vital_signs.loc[:,~vital_signs.columns.isin(["hadm_id", "subject_id", "stay_id", "charttime"])].isna()
print(f"NaNs :\n{is_nan.sum()}")
vital_signs = vital_signs[~is_nan.all(axis=1)]
display(vital_signs)

NaNs :
dicom_id            0
heart_rate     315233
sbp            332300
dbp            332408
mbp            311414
sbp_ni         636955
dbp_ni         637034
mbp_ni         637075
resp_rate      307789
temperature    838460
spo2           324767
glucose        898993
dtype: int64


,hadm_id,dicom_id,subject_id,stay_id,charttime,heart_rate,sbp,dbp,mbp,sbp_ni,dbp_ni,mbp_ni,resp_rate,temperature,spo2,glucose
0,20004004,75c5a3a1-63e507e2-0a2ac1da-9992be70-32d732c8,10236621,31718519,2171-06-28 13:27:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,88.0,NaN
1,20004004,b0c4597b-cf51b6f4-ec4d686e-b0eea53b-242fe454,10236621,31718519,2171-06-28 13:27:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,88.0,NaN
2,20004004,b566db46-4f529b7e-30db229c-7f85cd4b-6f66d34e,10236621,31718519,2171-06-28 13:27:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,88.0,NaN
3,20004004,cd65a5f7-d10c6901-53c6b6f3-82601b13-afbdda25,10236621,31718519,2171-06-28 13:27:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,88.0,NaN
4,20004004,75c5a3a1-63e507e2-0a2ac1da-9992be70-32d732c8,10236621,31718519,2171-06-28 13:29:00,110.0,NaN,NaN,NaN,NaN,NaN,NaN,18.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1043415,29999498,3a5c83bc-f9ff2720-87d86d06-5803e93d-03b0a8a8,13478841,33874605,2188-05-14 13:00:00,98.0,145.0,65.0,84.0,145.0,65.0,84.0,15.0,NaN,95.0,NaN
1043416,29999498,3a5c83bc-f9ff2720-87d86d06-5803e93d-03b0a8a8,13478841,33874605,2188-05-14 14:00:00,98.0,147.0,63.0,83.0,147.0,63.0,83.0,17.0,37.61,95.0,NaN
1043417,29999498,3a5c83bc-f9ff2720-87d86d06-5803e93d-03b0a8a8,13478841,33874605,2188-05-14 14:59:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,113.0
1043418,29999498,3a5c83bc-f9ff2720-87d86d06-5803e93d-03b0a8a8,13478841,33874605,2188-05-14 15:00:00,108.0,152.0,78.0,94.0,152.0,78.0,94.0,17.0,NaN,96.0,NaN


In [38]:
try:
    vital_signs = vital_signs.set_index("dicom_id")
except KeyError:
    pass
identity = lambda x: x
aggdict = {
    "hadm_id": identity,
    "subject_id": identity,
    "stay_id": identity,
    "charttime": identity,
    "heart_rate": [identity, lambda x: x],
}
vital_signs_fin = pd.DataFrame(index=vital_signs.index.unique())

for column in (set(vital_signs.columns) - {"hadm_id", "subject_id", "stay_id", "charttime"}):
    df_ = vital_signs[column].groupby(level=0).agg([
        "first",
        "last",
        np.nanmedian,
        np.nanmean,
        np.nanmax,
        np.nanmin
    ])
    vital_signs_fin[[f"{column}_{_}" for _ in df_.columns]] = df_
display(vital_signs_fin)
vital_signs_fin.to_csv("out/vital_signs.csv")

,glucose_first,glucose_last,glucose_nanmedian,glucose_nanmean,glucose_nanmax,glucose_nanmin,spo2_first,spo2_last,spo2_nanmedian,spo2_nanmean,...,heart_rate_nanmedian,heart_rate_nanmean,heart_rate_nanmax,heart_rate_nanmin,temperature_first,temperature_last,temperature_nanmedian,temperature_nanmean,temperature_nanmax,temperature_nanmin
dicom_id,,,,,,,,,,,,,,,,,,,,,
75c5a3a1-63e507e2-0a2ac1da-9992be70-32d732c8,110.0,109.0,109.5,109.5,110.0,109.0,88.0,100.0,100.0,98.833333,...,106.5,101.333333,118.0,82.0,36.56,36.89,36.78,36.743333,36.89,36.56
b0c4597b-cf51b6f4-ec4d686e-b0eea53b-242fe454,110.0,109.0,109.5,109.5,110.0,109.0,88.0,100.0,100.0,98.833333,...,106.5,101.333333,118.0,82.0,36.56,36.89,36.78,36.743333,36.89,36.56
b566db46-4f529b7e-30db229c-7f85cd4b-6f66d34e,110.0,109.0,109.5,109.5,110.0,109.0,88.0,100.0,100.0,98.833333,...,106.5,101.333333,118.0,82.0,36.56,36.89,36.78,36.743333,36.89,36.56
cd65a5f7-d10c6901-53c6b6f3-82601b13-afbdda25,110.0,109.0,109.5,109.5,110.0,109.0,88.0,100.0,100.0,98.833333,...,106.5,101.333333,118.0,82.0,36.56,36.89,36.78,36.743333,36.89,36.56
dfdeffcc-a8c1e2ba-c754f836-4232a37a-2fcc14a3,99.0,99.0,99.0,99.0,99.0,99.0,100.0,98.0,97.5,97.071429,...,76.5,77.428571,99.0,62.0,36.83,37.06,37.06,36.983333,37.06,36.83
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
211b69be-01ae57d7-0774634f-18585752-f5d74b64,76.0,76.0,76.0,76.0,76.0,76.0,97.0,93.0,96.0,95.285714,...,94.0,94.142857,101.0,87.0,37.11,36.67,37.22,37.125000,37.39,36.67
3c322b14-20d11cae-99cc5bb7-24893b03-ffe0be40,76.0,76.0,76.0,76.0,76.0,76.0,97.0,93.0,96.0,95.285714,...,94.0,94.142857,101.0,87.0,37.11,36.67,37.22,37.125000,37.39,36.67
4f7876f1-97bbbb4b-20101585-3fdaa080-706db488,76.0,76.0,76.0,76.0,76.0,76.0,97.0,93.0,96.0,95.285714,...,94.0,94.142857,101.0,87.0,37.11,36.67,37.22,37.125000,37.39,36.67


## Now for lab tests

In [40]:
lab_data = run_query(f"""
select
    a.subject_id,
    a.hadm_id,
    b.dicom_id,
    REPLACE ( CAST ( DATE ( a.charttime ) AS STRING ), '-', '' ) as date,
    REPLACE ( CAST ( TIME ( a.charttime ) AS STRING ), ':', '' ) as charttime,
    a.valuenum,
    items.label
FROM `physionet-data.mimic_icu.icustays` icu
JOIN `physionet-data.mimic_hosp.labevents` a
ON icu.hadm_id = a.hadm_id
    AND icu.hadm_id IN {tuple(sample.hadm_id.values)}
JOIN `physionet-data.mimic_cxr.record_list` b
ON a.subject_id = b.subject_id
JOIN `physionet-data.mimic_cxr.dicom_metadata_string` d
ON b.dicom_id = d.dicom
    AND {time_query(period = 1, time_col = "charttime")}
JOIN `physionet-data.mimic_hosp.d_labitems` items
ON a.itemid = items.itemid
ORDER BY a.hadm_id, a.charttime
""")
lab_data = lab_data[lab_data["valuenum"].notna()]
display(lab_data)

,subject_id,hadm_id,dicom_id,date,charttime,valuenum,label
0,10236621,20004004,75c5a3a1-63e507e2-0a2ac1da-9992be70-32d732c8,21710628,143600,15.0,Bicarbonate
1,10236621,20004004,75c5a3a1-63e507e2-0a2ac1da-9992be70-32d732c8,21710628,143600,4.4,Potassium
2,10236621,20004004,b0c4597b-cf51b6f4-ec4d686e-b0eea53b-242fe454,21710628,143600,1.0,Creatinine
3,10236621,20004004,b566db46-4f529b7e-30db229c-7f85cd4b-6f66d34e,21710628,143600,4.4,Potassium
4,10236621,20004004,cd65a5f7-d10c6901-53c6b6f3-82601b13-afbdda25,21710628,143600,2.8,Phosphate
...,...,...,...,...,...,...,...
6521109,13478841,29999498,3c1792b1-c013c47a-1f4ddffc-f2b61b43-0951ce17,21880519,225100,8.0,Alanine Aminotransferase (ALT)
6521111,13478841,29999498,3c1792b1-c013c47a-1f4ddffc-f2b61b43-0951ce17,21880519,225100,30.5,MCH
6521113,13478841,29999498,3c1792b1-c013c47a-1f4ddffc-f2b61b43-0951ce17,21880519,225100,0.0,Myelocytes
6521114,13478841,29999498,3c1792b1-c013c47a-1f4ddffc-f2b61b43-0951ce17,21880519,225100,2.0,Nucleated Red Cells


In [41]:
idxs, high_idxs, low_idxs = get_static_data_index(
    static_data=lab_data,
    N_measr = 100,
    N_unique = 12,
    N_count = 5,
    N_dicom = 5
)
print(f"Count, high-count measurements: {len(high_idxs)}\nCount, low-count measurements: {len(low_idxs)}")
lab_data[lab_data.label.isin(idxs)]

Count, high-count measurements: 141
Count, low-count measurements: 61


,subject_id,hadm_id,dicom_id,date,charttime,valuenum,label
0,10236621,20004004,75c5a3a1-63e507e2-0a2ac1da-9992be70-32d732c8,21710628,143600,15.0,Bicarbonate
1,10236621,20004004,75c5a3a1-63e507e2-0a2ac1da-9992be70-32d732c8,21710628,143600,4.4,Potassium
2,10236621,20004004,b0c4597b-cf51b6f4-ec4d686e-b0eea53b-242fe454,21710628,143600,1.0,Creatinine
3,10236621,20004004,b566db46-4f529b7e-30db229c-7f85cd4b-6f66d34e,21710628,143600,4.4,Potassium
4,10236621,20004004,cd65a5f7-d10c6901-53c6b6f3-82601b13-afbdda25,21710628,143600,2.8,Phosphate
...,...,...,...,...,...,...,...
6521109,13478841,29999498,3c1792b1-c013c47a-1f4ddffc-f2b61b43-0951ce17,21880519,225100,8.0,Alanine Aminotransferase (ALT)
6521111,13478841,29999498,3c1792b1-c013c47a-1f4ddffc-f2b61b43-0951ce17,21880519,225100,30.5,MCH
6521113,13478841,29999498,3c1792b1-c013c47a-1f4ddffc-f2b61b43-0951ce17,21880519,225100,0.0,Myelocytes
6521114,13478841,29999498,3c1792b1-c013c47a-1f4ddffc-f2b61b43-0951ce17,21880519,225100,2.0,Nucleated Red Cells


In [43]:
nan_count_lab = lab_data.valuenum.isna().sum()
print(f"{nan_count_lab} ({100*nan_count_lab/len(lab_data)}%) NaNs")

aggfuncs =  [
    "first",
    "last",
    np.nanmedian,
    np.nanmean,
    np.nanmax,
    np.nanmin,
]
num_val_lab = lab_data.pivot_table(
    index=['dicom_id'],
    columns=['label'],
    values='valuenum',
    aggfunc=aggfuncs
)
uniques = lab_data.pivot_table(columns=['label'], values='valuenum', aggfunc=lambda x: len(x.unique())).loc["valuenum"]

cutoff_percent = .00
idx = num_val_lab["nanmedian"].apply(lambda col: True if (~col.isna()).sum() >= int(len(num_val_lab["nanmedian"])*cutoff_percent) else False, axis=0)

new_lab_df = []
for parameter in aggfuncs:
    if type(parameter) is not str:
        name = parameter.__name__
    else:
        name = parameter
    _ = num_val_lab[name].loc[:,num_val_lab[name].columns.isin(idx[idx].index)]
    _.columns = pd.Series(_.columns).apply(lambda x: "_".join((name, x)))
    new_lab_df.append(_)
new_lab_df = pd.concat(new_lab_df, axis=1)

display(new_lab_df)
new_lab_df.to_csv("out/labevents.csv")

0 (0.0%) NaNs


label,first_% Hemoglobin A1c,first_24 hr Calcium,first_24 hr Creatinine,first_24 hr Protein,first_25-OH Vitamin D,first_ARCH-1,first_Absolute Basophil Count,first_Absolute CD3 Count,first_Absolute CD4 Count,first_Absolute CD8 Count,...,nanmin_WBC Casts,nanmin_WBC Count,nanmin_Waxy Casts,nanmin_White Blood Cells,nanmin_Young Cells,nanmin_eAG,nanmin_pCO2,nanmin_pH,nanmin_pO2,nanmin_tacroFK
dicom_id,,,,,,,,,,,,,,,,,,,,,
0003494b-20c32dda-59f01833-783fbd4b-8d48e435,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,5.2,NaN,NaN,NaN,5.50,NaN,NaN
000512db-61f1b4e0-5b16d40e-f6aeb922-21f71de8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,18.2,NaN,NaN,NaN,NaN,NaN,NaN
00054ff1-47f5c5c2-e8dc7c52-1deafa05-e1174cd0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,8.7,NaN,NaN,41.0,6.00,48.0,NaN
00070613-ea444275-19917990-3a18440d-0c68318e,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,11.3,NaN,NaN,30.0,7.31,82.0,NaN
00094318-ffb6a7d8-6dd1e667-280271ad-ae663064,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,6.3,NaN,NaN,NaN,5.00,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
fff4ed16-e7de1438-96706a1f-7839fce2-cc4cbc48,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,6.6,NaN,NaN,46.0,7.41,94.0,NaN
fff72ced-b8325d4f-6108dc57-815a33ca-cf929495,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,13.7,NaN,NaN,48.0,5.00,77.0,NaN
fff85c15-2446d444-26d09a19-cab887d9-adec07c3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,7.1,NaN,NaN,NaN,5.00,NaN,NaN
